<a href="https://colab.research.google.com/github/j03m/lstm-price-predictor/blob/main/Coin_Predictions_close.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IMPORT DATASETS AND LIBRARIES


In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import plotly.express as px
from copy import copy
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, confusion_matrix, classification_report, accuracy_score, f1_score
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
import requests
from requests.exceptions import HTTPError
import json as js
from datetime import datetime, timedelta 
import time
from os.path import exists
from decimal import *


#Library

In [3]:
# Function to plot interactive plots using Plotly Express
sc = MinMaxScaler()
num_features = 3
coin_base = False
ku_coin = True
COINBASE_REST_API = 'https://api.pro.coinbase.com' 
COINBASE_PRODUCTS = COINBASE_REST_API+'/products'
KUCOIN_REST_API = "https://api.kucoin.com"
KUCOIN_PRODUCTS = KUCOIN_REST_API+ "/api/v1/market/allTickers"
KUCOIN_CANDLES = KUCOIN_REST_API+ "/api/v1/market/candles"

data_path = '/content/drive/My Drive/output.csv'
model_path = "/content/drive/My Drive/model_ohlc.h5"

def interactive_plot(df, title):
  fig = px.line(title = title)
  for i in df.columns[1:]:
    fig.add_scatter(x = df['Date'], y = df[i], name = i)
  fig.show()

def get_single_stock(price_df, vol_df, name):
    return pd.DataFrame({'Date': price_df['Date'], 'Close': price_df[name], 'Volume': vol_df[name]})

def scale_data(data):
  # Scale the data
  scaled_data = sc.fit_transform(data)
  return scaled_data

def sort_date(pric_df):
  pric_df = pric_df.sort_values(by = ['Date'])
  return pric_df

def append_price_dif(df):
  df['Target'] = df['Close'].shift(-1)
  df['Diff'] = df['Target'] - df['Close']
  df = df[:-1]
  return df

def append_price_dif_(df):
  df['Target'] = df['Close'].shift(-1)
  df['Diff'] = df['Target'] - df['Close']
  return df

def append_15d_slope(df):
  df['15Close'] = df['Close'].shift(15)
  df['15Date'] = df['Date'].shift(15)
  df['Trend'] = (df['Close'] - df['15Close']) / 15
  df = df[15:]
  return df

def show_plot(data, title):
  plt.figure(figsize = (13, 5))
  plt.plot(data, linewidth = 3)
  plt.title(title)
  plt.grid()

def build_model(features, outcomes):
  # Create the model
  inputs = keras.layers.Input(shape=(features,outcomes))
  x = keras.layers.LSTM(150, return_sequences= True)(inputs)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150, return_sequences=True)(x)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150)(x)
  outputs = keras.layers.Dense(1, activation='linear')(x)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer='adam', loss="mse")
  return model

def connect(url, params):   
  response = requests.get(url,params)
  response.raise_for_status()
  return response

def coinbase_json_to_df(delta, product, granularity='86400'):
  start_date = (datetime.today() - timedelta(seconds=delta*int(granularity))).isoformat()
  end_date = datetime.now().isoformat()
  # Please refer to the coinbase documentation on the expected parameters
  params = {'start':start_date, 'end':end_date, 'granularity':granularity}
  response = connect(COINBASE_PRODUCTS+'/' + product + '/candles', params)
  response_text = response.text
  df_history = pd.read_json(response_text)
  # Add column names in line with the Coinbase Pro documentation
  df_history.columns = ['time','low','high','open','close','volume']
  df_history['time'] = [datetime.fromtimestamp(x) for x in df_history['time']]
  return df_history

def ku_coin_json_to_df(delta, product, granularity='86400'):
  granularity = int(granularity)
  start_date = (datetime.today() - timedelta(seconds=delta*granularity))
  end_date = datetime.now()

  # Please refer to the kucoin documentation on the expected parameters
  params = {'startAt':int(start_date.timestamp()), 'endAt':int(end_date.timestamp()), 'type':gran_to_string(granularity), 'symbol':product}
  response = connect(KUCOIN_CANDLES, params)
  response_text = response.text
  response_data = js.loads(response_text);
  if (response_data["code"] != "200000"):
    raise Exception("Illegal response: " + response_text)
  
  df_history = pd.DataFrame(response_data["data"])

  # kucoin is weird in that they don't have candles for everything. IF we don't have the requested
  # number of bars here, it throws off the whole algo. I don't want to try and project so we 
  # just won't trade those instruments
  got_bars = len(df_history)
  if ( got_bars < delta):
    raise Exception("Requested:" + str(delta) + " bars " + " but only got:" + str(got_bars))

  df_history.columns = ['time','open','close','high','low','volume', 'amount']
  df_history['time'] = [datetime.fromtimestamp(int(x)) for x in df_history['time']]
  df_history['open'] = [float(x) for x in df_history['open']]
  df_history['close'] = [float(x) for x in df_history['close']]
  df_history['high'] = [float(x) for x in df_history['high']]
  df_history['low'] = [float(x) for x in df_history['low']]
  df_history['low'] = [float(x) for x in df_history['low']]
  df_history['volume'] = [float(x) for x in df_history['volume']]
  df_history['amount'] = [float(x) for x in df_history['amount']]
  return df_history
  
def gran_to_string(granularity):
  #todo implement this actually
  return "1day"

def get_coin_data_frames(time, product, granularity='86400'):
  if coin_base:
    df_raw = coinbase_json_to_df(time, product, granularity)
  else:
    df_raw = ku_coin_json_to_df(time, product, granularity)

  df_btc_history = df_raw
  if len(df_btc_history.index) == 0:
    print("No data for ", product)
  
  df_btc_history = df_btc_history.rename(columns={"time":"Date", "close":"Close", "volume":"Volume"})
  df_btc_history = sort_date(df_btc_history)
  df_btc_history = df_btc_history.drop(columns={"high", "low", "open"})
  df_btc_history = append_price_dif_(df_btc_history)
  df_btc_history = append_15d_slope(df_btc_history)
  df_btc_features = df_btc_history[["Close", "Volume", "Trend"]]
  df_history_scaled = sc.fit_transform(df_btc_features)
  return [df_btc_history, df_btc_features, df_history_scaled, df_raw]

def build_profit_estimate(predicted, df_btc_history):
  df_predicted_chart = pd.DataFrame();
  df_predicted_chart["Date"] = df_btc_history["Date"]
  df_predicted_chart["Predicted"] = predicted
  df_predicted_chart["Predicted-Target"] = df_predicted_chart["Predicted"].shift(-1)
  df_predicted_chart["Predicted-Diff"] = df_predicted_chart["Predicted-Target"] - df_predicted_chart["Predicted"]
  df_predicted_chart["Should-Trade"] = np.where(df_predicted_chart["Predicted-Diff"] > 0, True, False)
  df_predicted_chart["RealDiff"] = df_btc_history["Diff"]
  df_predicted_chart["Percent"] = df_predicted_chart["RealDiff"] / df_btc_history["Close"]
  df_predicted_chart["Profit"] = np.where(df_predicted_chart["Should-Trade"] > 0, df_predicted_chart["Percent"] * budget, 0)
  profit = df_predicted_chart["Profit"].sum()
  return [df_predicted_chart, profit]

def debug_prediction_frame(predicted, df_history, df_history_scaled):
  df_predicted_chart = pd.DataFrame();
  df_predicted_chart["Date"] = df_history["Date"]
  df_predicted_chart["Predicted"] = predicted
  df_predicted_chart["Original"] = df_history_scaled[:,0]
  df_predicted_chart["Original-Target"] = df_history_scaled[:,2]
  df_predicted_chart["Target-Date"] = df_predicted_chart["Date"].shift(-1)
  df_predicted_chart["Predicted-Diff"] = df_predicted_chart["Predicted"] - df_predicted_chart["Original"]
  df_predicted_chart["Actual-Diff"] = df_predicted_chart["Original-Target"] - df_predicted_chart["Original"]
  df_predicted_chart["Should-Trade"] = np.where(df_predicted_chart["Predicted-Diff"] > 0, True, False)
  df_predicted_chart["Close"] = df_history["Close"]
  df_predicted_chart["Target"] = df_history["Target"]
  df_predicted_chart["RealDiff"] = df_history["Diff"]
  df_predicted_chart["Percent"] = df_predicted_chart["RealDiff"] / df_predicted_chart["Close"]
  df_predicted_chart["Profit"] = np.where(df_predicted_chart["Should-Trade"] > 0, df_predicted_chart["Percent"] * budget, 0)
  return df_predicted_chart

def get_all_products():
  if coin_base:
    return get_all_coinbase_products()
  
  if ku_coin:
    return get_all_kucoin_products()

def get_all_kucoin_products():
  response = connect(KUCOIN_PRODUCTS, {})
  products = js.loads(response.text)
  df_products = pd.DataFrame(products["data"]["ticker"])
  df_products = df_products.rename(columns={"symbol":"id"})
  return df_products

def get_all_coinbase_products():
  response = connect(COINBASE_PRODUCTS, {})
  response_text = response.text
  df_products = pd.read_json(response_text)
  return df_products

def predict_trade(product, bars):
  [df_full, df_features, npa_scaled, df_raw] = get_coin_data_frames(bars, product)
  predicted = model.predict(npa_scaled).flatten()
  
  #convert to data frames that have the correct shape for being unscaled
  df_scaled = pd.DataFrame(npa_scaled, columns = ["Close", "Volume", "Trend"])
  
  # I want to believe that scaling happens on a per column basis, we only care about
  # price here so we will dummy out volume and trend and use the scaler on it
  # this kinda sucks, if we add features we'll need to add them here for unscaling
  df_temp = pd.DataFrame(predicted, columns = ["Close"])
  df_temp["Volume"] = 0
  df_temp["Trend"] = 0
  
  # unscale them both
  df_temp = pd.DataFrame(sc.inverse_transform(df_temp), columns = ["Close", "Volume", "Trend"])
  df_trade = pd.DataFrame(sc.inverse_transform(df_scaled), columns = ["Close", "Volume", "Trend"])
  
  # add predicted
  df_trade["Predicted"] = df_temp["Close"]
  df_trade = df_trade.tail(1)

  # add the product, derive a move and percent
  df_trade["Product"] = row.id;
  df_trade["Move"] = df_trade["Predicted"] - df_trade["Close"]
  df_trade["Percent"] = (df_trade["Move"] / df_trade["Close"]) * 100
  df_trade["RawPercent"] = df_trade["Move"] / df_trade["Close"]
  df_trade["250Fees"] = (250 * 0.004) * 2
  df_trade["5kFees"] = (5000 * 0.004) * 2
  df_trade["10kFees"] = (10000 * 0.0025) * 2
  df_trade["250Profit"] = (250 * df_trade["RawPercent"]) - df_trade["250Fees"] 
  df_trade["5kProfit"] = (5000 * df_trade["RawPercent"]) - df_trade["5kFees"]
  df_trade["10k0Profit"] = (10000 * df_trade["RawPercent"]) - df_trade["10kFees"]
  return df_trade

def get_training_set_for(ticker):
  target_df = get_single_stock(all_stocks_price_df, all_stocks_vol_df, ticker)
  target_df = append_price_dif(target_df)
  target_df = append_15d_slope(target_df)
  features = target_df[["Close", "Volume", "Trend", "Target"]]
  scaled_features = scale_data(features)
  X = []
  y = []
  for i in range(0, len(target_df)):
    X.append(scaled_features [i][0:num_features])
    y.append(scaled_features [i][num_features])
  
  X = np.asarray(X)
  y = np.asarray(y)
  return [scaled_features, X, y]
  
def train_model(model, X, y):

  # One day we might need test, but for now we don't we can use another
  # time series, we have so many
  # Split the data
  #split = int(0.7 * len(X))
  #X_train = X[:split]
  #y_train = y[:split]
  #X_test = X[split:]
  #y_test = y[split:]

  # Reshape the 1D arrays to 3D arrays to feed in the model
  X_train = np.reshape(X, (X.shape[0], X.shape[1], 1))
  
  history = model.fit(
      X_train, y,
      epochs = 20,
      batch_size = 32,
      validation_split = 0.2
  )
  return [model, history]

#pull training data 
all_stocks_price_df = sort_date(pd.read_csv('/content/drive/My Drive/Colab Notebooks/stock.csv'))
all_stocks_vol_df = sort_date(pd.read_csv("/content/drive/My Drive/Colab Notebooks/stock_volume.csv"))

# Get or Train a Model

In [4]:
model = None
file_exists = exists(model_path)

if file_exists:
  print("hello")
  model = keras.models.load_model(model_path)
else:
  print("nope")
  [scaled_features, X, y] = get_training_set_for("sp500")  
  model = build_model(num_features, 1)
  [model, history] = train_model(model, X, y)


hello


In [5]:
# additional training?
if 0:
  [scaled_features, X, y] = get_training_set_for("IBM")  
  [model, history] = train_model(model, X, y)


# Visualize and Backtest

In [16]:
budget = 5000
[btc_history, df_btc_features, df_history_scaled, df_raw] = get_coin_data_frames(30, "DOGE3L-USDT")
predicted = model.predict(df_history_scaled).flatten()
[df_profit, profit] = build_profit_estimate(predicted, btc_history)
df_chart = debug_prediction_frame(predicted, btc_history, df_history_scaled)
interactive_plot(df_chart[["Date","Original", "Predicted"]], "Wtf")
print("Profit:", profit)
df_chart


1/1 [==============================] - 0s 42ms/step


Profit: 2162.5610576982017


,Date,Predicted,Original,Original-Target,Target-Date,Predicted-Diff,Actual-Diff,Should-Trade,Close,Target,RealDiff,Percent,Profit
14,2022-12-12,0.937005,0.950596,0.794475,2022-12-13,-0.013592,-1.561211e-01,False,0.001006,0.001035,0.000029,0.028827,0.000000
13,2022-12-13,0.984286,1.000000,1.000000,2022-12-14,-0.015714,-2.220446e-16,False,0.001035,0.000929,-0.000106,-0.102415,0.000000
12,2022-12-14,0.816543,0.819421,0.614365,2022-12-15,-0.002877,-2.050561e-01,False,0.000929,0.000819,-0.000110,-0.118407,0.000000
11,2022-12-15,0.627001,0.632027,0.214365,2022-12-16,-0.005026,-4.176626e-01,False,0.000819,0.000565,-0.000254,-0.310134,0.000000
10,2022-12-16,0.206177,0.199319,0.228729,2022-12-17,0.006858,2.941071e-02,True,0.000565,0.000636,0.000071,0.125664,628.318584
9,2022-12-17,0.320802,0.320273,0.278453,2022-12-18,0.000529,-4.181953e-02,True,0.000636,0.000612,-0.000024,-0.037736,-188.679245
8,2022-12-18,0.271136,0.279387,0.401105,2022-12-19,-0.008251,1.217183e-01,False,0.000612,0.000448,-0.000164,-0.267974,0.000000
7,2022-12-19,0.033862,0.000000,0.000000,2022-12-20,0.033862,0.000000e+00,True,0.000448,0.000511,0.000063,0.140625,703.125000
6,2022-12-20,0.129320,0.107325,0.206630,2022-12-21,0.021995,9.930445e-02,True,0.000511,0.000482,-0.000029,-0.056751,-283.757339
5,2022-12-21,0.075613,0.057922,0.250829,2022-12-22,0.017691,1.929071e-01,True,0.000482,0.000542,0.000060,0.124481,622.406639


In [7]:
print("rscore: ", r2_score(predicted, df_history_scaled[:, 2]))



rscore:  0.14099733238475198


# Which coins are most profitable based on the above trading signals?

In [8]:
# download all known products and check who has the highest profit in 90 days
"""
# Fetch the top 10 and see if they predict up
df_products = get_all_products()
df_products = df_products[df_products.id.str.endswith('USD')]
df_products = df_products[df_products.trading_disabled == False]
df_products = df_products[df_products.cancel_only == False]
df_profit = pd.DataFrame();
df_profit["Product"] = [];
df_profit["Profit"] = [];
for index, row in df_products.iterrows():
  try:
    print("fetching: ", row.id)
    [df_full, df_features, df_scaled] = get_coin_data_frames_test(90, row.id)
    predicted = model.predict(df_scaled.flatten())
    [df_chart, profit] = build_profit_estimate(predicted, df_full)
    df_profit.loc[len(df_profit.index)] = [row.id, profit] 
  except Exception as inst:
    print("Error: ", inst)
  time.sleep(1)

df_profit
"""

'\n# Fetch the top 10 and see if they predict up\ndf_products = get_all_products()\ndf_products = df_products[df_products.id.str.endswith(\'USD\')]\ndf_products = df_products[df_products.trading_disabled == False]\ndf_products = df_products[df_products.cancel_only == False]\ndf_profit = pd.DataFrame();\ndf_profit["Product"] = [];\ndf_profit["Profit"] = [];\nfor index, row in df_products.iterrows():\n  try:\n    print("fetching: ", row.id)\n    [df_full, df_features, df_scaled] = get_coin_data_frames_test(90, row.id)\n    predicted = model.predict(df_scaled.flatten())\n    [df_chart, profit] = build_profit_estimate(predicted, df_full)\n    df_profit.loc[len(df_profit.index)] = [row.id, profit] \n  except Exception as inst:\n    print("Error: ", inst)\n  time.sleep(1)\n\ndf_profit\n'

# What has a buy indicator for tomorrow?

In [9]:
# Fetch the top 10 and see if they predict up
df_products = get_all_products()
df_products = df_products[df_products.id.str.endswith('USDT')]

if coin_base:
  df_products = df_products[df_products.trading_disabled == False]
  df_products = df_products[df_products.cancel_only == False]

df_trades = pd.DataFrame();
bars = 90
counter = 0;
for index, row in df_products.iterrows():
  try:
    print("fetching: ", row.id)
    
    df_trade = predict_trade(row.id, bars)
   
    # we need to unscale the predicted values so that we have an entry and exit point
    # entry should be roughly close and exit should be roughly predicted

    # Stick this on the end of the main dataframe
    df_trades = df_trades.append(df_trade);
    
    #counter+=1
    #if counter > 5:
    #  break
  except Exception as inst:
    #raise inst
    print("Error: ", inst)
  time.sleep(2)
df_trades.reset_index()
df_buys = df_trades[df_trades['Move'] > 0] 
df_shorts = df_trades[df_trades['Move'] < 0] 




fetching:  NKN-USDT
3/3 [==============================] - 0s 7ms/step
fetching:  GEM-USDT
Error:  Requested:90 bars  but only got:77
fetching:  CUSD-USDT
3/3 [==============================] - 0s 7ms/step
fetching:  LTC3L-USDT


KeyboardInterrupt: ignored

In [ ]:
df_shorts

In [ ]:
with open(data_path, 'w', encoding = 'utf-8-sig') as f:
  df_trades.to_csv(f)

In [ ]:
model.save(model_path)

In [17]:
predict_trade("DOGE3L-USDT", 90)

3/3 [==============================] - 0s 9ms/step


,Close,Volume,Trend,Predicted,Product,Move,Percent,RawPercent,250Fees,5kFees,10kFees,250Profit,5kProfit,10k0Profit
74,0.000512,1.886607e+08,-0.000039,0.000559,LTC3L-USDT,0.000047,9.26664,0.092666,2.0,40.0,50.0,21.1666,423.331995,876.663991
